In [75]:
from mp_api.client import MPRester
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter, PourbaixEntry


API_KEY = "M3iWLItVErrMW5qgPwe5tzZFxjlROt1x"  

In [76]:
materials = ["mp-2657", "mp-555487"]
mp_entries = []

with MPRester(API_KEY) as mpr:
    for mp_id in materials:
        material_entries = mpr.materials.search(material_ids=[mp_id])

        if material_entries:
            mp_entries.append(material_entries[0])

chemsys_list = [entry.chemsys for entry in mp_entries if hasattr(entry, 'chemsys')]
composition_list = [entry.composition_reduced for entry in mp_entries if hasattr(entry, 'composition_reduced')]
materials_gga = [material + "-GGA" for material in materials]

print(chemsys_list)
print(composition_list)
print(materials_gga)

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

['O-Ti', 'O-Sn']
[Composition('Ti1 O2'), Composition('Sn1 O2')]
['mp-2657-GGA', 'mp-555487-GGA']


In [77]:
pbx_data = []
pbx_entries = []
pbx_plots = []

with MPRester(API_KEY) as mpr:
    for i in chemsys_list:
        pourbaix_entries = mpr.get_pourbaix_entries(i)
        pbx_entries.append(pourbaix_entries)
        pbx_diagram = PourbaixDiagram(pourbaix_entries)
        pbx_data.append(pbx_diagram)  
        pbx_plotter = PourbaixPlotter(pbx_diagram)
        pbx_plots.append(pbx_plotter)


Retrieving ThermoDoc documents:   0%|          | 0/218 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/124 [00:00<?, ?it/s]

In [78]:
matched_entries = []

for mat_id, entries in zip(materials_gga, pbx_entries):
    matched_entry = next((entry for entry in entries if entry.entry_id == mat_id), None)
    
    if matched_entry:
        matched_entries.append(matched_entry)
    else:
        matched_entries.append(None)

print(matched_entries)

[PourbaixEntry(Ti2 O4 with energy = -9.6864, npH = -8.0, nPhi = -8.0, nH2O = 4.0, entry_id = 'mp-2657-GGA'), PourbaixEntry(Sn12 O24 with energy = -8.8371, npH = -48.0, nPhi = -48.0, nH2O = 24.0, entry_id = 'mp-555487-GGA')]


In [81]:
stabilities = []

for i, x in zip(pbx_data, matched_entries):
    stability = i.get_decomposition_energy(x, 7, 0)
    stabilities.append(stability)

print(stabilities)


[0.037221306666667196, 0.011987455277777078]
